In [ ]:
#pip install pandas-profiling

In [ ]:
import sklearn
sklearn. __version__ 

In [ ]:
import numpy as np

from pandas_profiling import ProfileReport

In [ ]:
import pickle
import pandas as pd
# pd.set_option('display.max_rows', 2000)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 10000)


In [ ]:
# df_review = pd.read_pickle('reviews_all.pkl')
# df_review['review_body']

# Dataframe

In [ ]:
df_main = pd.read_pickle('df_main.pkl')
df_main.isnull().sum()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
 

s = df_main['vitolas']

mlb = MultiLabelBinarizer()

df_vitola = pd.DataFrame(mlb.fit_transform(s),columns= (f'vitola_' + mlb.classes_), index=df_main.index)

In [ ]:
%store df_vitola
#del df_vitola

In [ ]:
df_main_vitola = pd.concat([df_main, df_vitola], axis=1)

In [ ]:
df_KNN = df_main_vitola.copy()

In [ ]:
df_KNN.head()

In [ ]:
df_KNN1 = df_KNN.drop(['vitolas', 'Shapes','prod_name', 'rating', 'rating_count', 'desc'], axis=1)

# DF to Encode

In [ ]:
df_KNN1.head()

In [ ]:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
#df_KNN1.to_pickle('KNN1.pkl')

In [ ]:
encoder = OrdinalEncoder()
imputer = IterativeImputer(ExtraTreesRegressor())
inv_tran = []
# create a list of categorical columns to iterate over
cat_cols = ['profile','Binder','Filler','Flavored','Has Tip','Origin','Pressed','Sweet','Wrapper', 'Brand:']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Encoder Dict
    inverse_trans = inv_tran.append(encoder.categories_)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(df_KNN1[columns])

In [ ]:

%store inv_tran
#del inv_tran

In [ ]:
cat_cols = ['profile','Binder','Filler','Flavored','Has Tip','Origin','Pressed','Sweet','Wrapper', 'Brand:']

categorical_columns = cat_cols
for i,column in enumerate(categorical_columns):
    label_list = inv_tran[i][0]
    df[column] = [label_list[j] for j in df[column]]
    print(column)

In [ ]:
df

In [ ]:
#df.to_pickle('impute.pkl') 

In [ ]:
df_impute = pd.read_pickle('impute.pkl')
df_impute

# Verify

In [ ]:
df_verify = pd.concat([df_main, df_impute], axis=1)

In [ ]:
df_verify[['prod_name', 'Filler']]

In [ ]:
# impute data and convert 
encode_data = pd.DataFrame(np.round(imputer.fit_transform(df_KNN1)),columns = df_KNN1.columns)

# Encode Data

In [ ]:
# impute data and convert 
encode_data[list(cat_cols)] = encode_data[list(cat_cols)].astype(int)


#encode_data.dtypes
df = encode_data[cat_cols]

In [ ]:
df

In [ ]:
df9 = pd.DataFrame(encoder.categories_)
df9

In [ ]:
encode_data.isnull().sum()

In [ ]:
df_main.isnull().sum()


In [ ]:
df_main['rating'].fillna( 0, inplace=True)

In [ ]:
df_main['rating_count'].fillna( 0, inplace=True)

In [ ]:
df_main['Has Tip'].fillna( 'No', inplace=True)

# Is Null

In [ ]:
df_main.isnull().sum()

In [ ]:
import numpy as np
df_nobrand = df_main.drop('Brand:',1)
df_nobrand['is_null'] = df_nobrand.isna().sum(axis=1).apply(lambda x: 0 if x==0 else 1)
df_nobrand['is_null'].sum()
df_main['is_null'] = df_main.isna().sum(axis=1).apply(lambda x: 0 if x==0 else 1)
df_main['is_null'].sum()

In [ ]:
df_nobrand = df_main.drop('Brand:',1)

In [ ]:
df_nobrand['is_null'] = df_nobrand.isna().sum(axis=1).apply(lambda x: 0 if x==0 else 1)

In [ ]:
df_nobrand['is_null'].sum()

In [ ]:
df_main['is_null'] = df_main.isna().sum(axis=1).apply(lambda x: 0 if x==0 else 1)

In [ ]:
df_main['is_null'].sum()

In [ ]:
df_main

# Null DF

In [ ]:
df1 = df_main[df_main['profile'].isnull()]
df1

In [ ]:
pd.DataFrame(df_main['Binder'].unique())

In [ ]:
import plotly.express as px
df = df_main
# Here we use a column with categorical data
fig = px.histogram(df, x="Binder")
fig.show()

# Profile Report

In [ ]:
#df_main.profile_report()

In [ ]:
# df_main['rating'] = [x.split(' ')[1] if x!=None else x for x in df_main['rating']]

In [ ]:
# df_main['rating_count'] = [x.split(' ')[3] if x!=None else x for x in df_main['rating_count']]

In [ ]:
df_main[["prod_name","desc"]]

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


In [ ]:
lemmatizer = WordNetLemmatizer()

# NLP process

In [ ]:
def nlp_process(desc):
    
    '''tokenize, lower, stopwords, remove punctuation, lemmatizer'''
    
    #tokenize
    tokens = nltk.word_tokenize(desc)
    #lower words
    lower_words=[word2.lower() for word2 in tokens]
    #remove stopwords
    stp_words = ([word3 for word3 in lower_words if word3 not in (stopwords.words('english'))])
    #remove punctuation
    text_without_punct = [s.translate(str.maketrans('', '', string.punctuation)) for s in stp_words]
    #lemmatize
    lemmatized_words=[lemmatizer.lemmatize(word=word,pos='v') for word in text_without_punct]
    lemmatizeddf= pd.DataFrame({'original_word': text_without_punct,'lemmatized_word': lemmatized_words})
    lemmatizeddf=lemmatizeddf[['original_word','lemmatized_word']]
    return lemmatizeddf

In [ ]:
nlp_process(df_main['desc'][1])

# Preview

In [ ]:
def preview(row):
    prod_name = df_main['prod_name'][row]
    desc = df_main['desc'][row]
    process = df_main['process_desc'][row]
    return display(prod_name), display(desc), display(process)

In [ ]:
#tokenize df

#df_main['process_desc'] = df_main.apply(lambda row: nlp_process(row['desc']), axis=1)

# df_main['desc'].head()

In [ ]:
preview(1)

In [ ]:
#tokenize df

#df_main['desc'] = df_main.apply(lambda row: nltk.word_tokenize(row['desc']), axis=1)

# df_main['desc'].head()

In [ ]:
df_main.head()

In [ ]:
# token = nltk.word_tokenize(df_main['desc'][0])
# token

In [ ]:
#lower

# texts=token
# lower_words=[word.lower() for word in texts]
# lower_words

In [ ]:
#stopwords

# text = lower_words
# stop_words = ([word for word in text if word not in (stopwords.words('english'))])
# stop_words

In [ ]:

display(f"Punctuation symbols: {string.punctuation}")

In [ ]:
#remove punctuation
# text_without_punct = [s.translate(str.maketrans('', '', string.punctuation)) for s in stop_words]
# text_without_punct = text.translate(str.maketrans('', '', string.punctuation))
# display(f"Text without punctuation: {text_without_punct}")

In [ ]:
#lemmatize trouble variations

# words=text_without_punct
# lemmatized_words=[lemmatizer.lemmatize(word=word,pos='v') for word in words]
# lemmatizeddf= pd.DataFrame({'original_word': words,'lemmatized_word': lemmatized_words})
# lemmatizeddf=lemmatizeddf[['original_word','lemmatized_word']]
# lemmatizeddf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df_main['prod_name']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())

tfid_df = pd.DataFrame(X)
df_vect = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
tfid_df

In [ ]:
df_vect = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
df_vect.shape

In [ ]:
#df_vect.drop([col for col, val in df_vect.mean().iteritems() if val < 0.0025], axis=1, inplace=True)

# df_vect

In [ ]:
df_vect

In [ ]:
#df_vect.to_pickle('prod_vect.pkl')

In [ ]:
# 2. only keep words w average tfidf scores over 0.0001
relevant = []
for word in df_vect.columns:
    if df_vect[word].mean() > 0.0025:
        relevant.append(df_vect[word])
len(relevant)

In [ ]:
relevant

In [ ]:
frames = [df_main, df_vect]

result = pd.concat(frames, axis=1)
result

In [ ]:
df_main["desc"]= df_main["desc"].astype(str) 


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS 

# Wordcloud

In [ ]:
## Make a list of stopwords to remove
from nltk.corpus import stopwords
import string

In [ ]:
stopwords_list = stopwords.words('english')
stopwords_list

In [ ]:
additional_words = ['cigar', 'year', 'back', 'time', 'one', 'limited', 'cigars', 'new', 'get'
                   ,'years', 'blend','good','La','Gurkha','like','made','Serie', 'classic', 'dark','name']

In [ ]:
## Add punctuation to stopwords_list
stopwords_list+=string.punctuation
## Add additional_words to stopwords_list
stopwords_list.extend(additional_words)
stopwords_list

# Wordcloud Reviews

# Wordcloud Product Description

In [ ]:

wordcloud = WordCloud(width=800, height=400, stopwords=stopwords_list, background_color="white", max_words=200, 
                      contour_width=3, 
                      contour_color='steelblue')

wordcloud.generate(df_main['desc'].to_string())

wordcloud.to_image()

In [ ]:
stemmer = PorterStemmer()
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [ ]:

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = df_main['desc']
# data.head()

tf_idf_vectorizor = TfidfVectorizer(stop_words = stopwords,#tokenizer = tokenize_and_stem,
                             max_features = 5000)
%time tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()
pd.DataFrame(tf_idf_array, columns=tf_idf_vectorizor.get_feature_names()).head()

In [ ]:
initial_centroids = np.random.permutation(tf_idf_array.shape[0])[:3]
initial_centroids
centroids = tf_idf_array[initial_centroids]
centroids.shape
dist_to_centroid =  pairwise_distances(tf_idf_array,centroids, metric = 'euclidean')
cluster_labels = np.argmin(dist_to_centroid, axis = 1)

In [ ]:
class Kmeans:
    """ K Means Clustering
    
    Parameters
    -----------
        k: int , number of clusters
        
        seed: int, will be randomly set if None
        
        max_iter: int, number of iterations to run algorithm, default: 200
        
    Attributes
    -----------
       centroids: array, k, number_features
       
       cluster_labels: label for each data point
       
    """
    
    def __init__(self, k, seed = None, max_iter = 200):
        self.k = k
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)
        self.max_iter = max_iter
        
            
    
    def initialise_centroids(self, data):
        """Randomly Initialise Centroids
        
        Parameters
        ----------
        data: array or matrix, number_rows, number_features
        
        Returns
        --------
        centroids: array of k centroids chosen as random data points 
        """
        
        initial_centroids = np.random.permutation(data.shape[0])[:self.k]
        self.centroids = data[initial_centroids]

        return self.centroids
    
    
    def assign_clusters(self, data):
        """Compute distance of data from clusters and assign data point
           to closest cluster.
        
        Parameters
        ----------
        data: array or matrix, number_rows, number_features
        
        Returns
        --------
        cluster_labels: index which minmises the distance of data to each
        cluster
            
        """
        
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        
        dist_to_centroid =  pairwise_distances(data, self.centroids, metric = 'euclidean')
        self.cluster_labels = np.argmin(dist_to_centroid, axis = 1)
        
        return  self.cluster_labels
    
    
    def update_centroids(self, data):
        """Computes average of all data points in cluster and
           assigns new centroids as average of data points
        
        Parameters
        -----------
        data: array or matrix, number_rows, number_features
        
        Returns
        -----------
        centroids: array, k, number_features
        """
        
        self.centroids = np.array([data[self.cluster_labels == i].mean(axis = 0) for i in range(self.k)])
        
        return self.centroids
    
    
    def convergence_calculation(self):
        """
        Calculates 
        
        """
        pass
    
    def predict(self, data):
        """Predict which cluster data point belongs to
        
        Parameters
        ----------
        data: array or matrix, number_rows, number_features
        
        Returns
        --------
        cluster_labels: index which minmises the distance of data to each
        cluster
        """
        
        return self.assign_clusters(data)
    
    def fit_kmeans(self, data):
        """
        This function contains the main loop to fit the algorithm
        Implements initialise centroids and update_centroids
        according to max_iter
        -----------------------
        
        Returns
        -------
        instance of kmeans class
            
        """
        self.centroids = self.initialise_centroids(data)
        
        # Main kmeans loop
        for iter in range(self.max_iter):

            self.cluster_labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)          
            if iter % 100 == 0:
                print("Running Model Iteration %d " %iter)
        print("Model finished running")
        return self

In [ ]:
from sklearn.datasets import make_blobs
# create blobs
data = make_blobs(n_samples=200, n_features=2, centers=4, cluster_std=1.6, random_state=50)
# create np array for data points
points = data[0]
# create scatter plot
plt.scatter(data[0][:,0], data[0][:,1], c=data[1], cmap='viridis')
plt.xlim(-15,15)
plt.ylim(-15,15)

X = data[0]
X[2]

In [ ]:
df_main.info()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from scipy.stats import multivariate_normal as mvn
import nltk
import os
import random




import string
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# email module has some useful functions
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')